In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [1]:
path = "/content/drive/MyDrive/Colab Notebooks/"

In [2]:
import numpy as np
import pandas as pd
from scipy.sparse import hstack
import scipy
import pickle
from sklearn.feature_extraction.text import CountVectorizer

import time
t=1

In [3]:
X = scipy.sparse.load_npz(path+'data/countvec/train_steps_vec.npz').toarray().astype(np.int32)
X_n = pd.read_csv(path+'data/recipe_train.csv').loc[:, ['n_steps', 'n_ingredients']].values
X = np.hstack([X, X_n])
Y = pd.read_csv(path+"data/recipe_train.csv")['duration_label']

In [4]:
vocab = pickle.load(open(path+"data/countvec/train_steps_countvectorizer.pkl", "rb"))
vocab_dict = vocab.vocabulary_

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [5]:
three = pd.read_csv(path+"data/three.csv")['steps']
three_n = pd.read_csv(path+"data/three.csv").loc[:, ['n_steps', 'n_ingredients']]

In [6]:
vectorizer = CountVectorizer(vocabulary=vocab_dict)
three = vectorizer.transform(three)

del vocab,  vocab_dict, vectorizer

In [7]:
three = three.toarray().astype(np.int32)
three = np.hstack([three, three_n.values])

In [9]:
del three_n, X_n

In [ ]:
X = np.vstack((X, three)).astype(np.int32)
Y = np.hstack((Y, np.array([3.0 for i in range(three.shape[0])])))

In [ ]:
from sklearn.model_selection import train_test_split
X, X_t, y, y_t = train_test_split(X, Y, test_size=0.2, random_state=42)
del three

# ------------------------------------------------------

In [ ]:
import gc
import os
import torch
from torch import nn
from torch.utils.data import Dataset, TensorDataset, DataLoader

if 1:
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
else:
    device = 'cpu'
print('Using {} device'.format(device))

Using cuda:0 device


# 2 inputs

In [ ]:
X = torch.tensor(X)
X_t = torch.tensor(X_t)

In [ ]:
y = torch.tensor(y)-1
y_t = torch.tensor(y_t)-1

l1 = X.shape[1]

print(l1)

17967


In [ ]:
training_data = TensorDataset(X, y)
testing_data = TensorDataset(X_t, y_t)

train_dataloader = DataLoader(training_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(testing_data, batch_size=32, shuffle=True)

tf1,train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {tf1.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([32, 17967])
Labels batch shape: torch.Size([32])


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, l1):
        super(NeuralNetwork, self).__init__()
      
        self.blackbox = nn.Sequential(
            nn.Linear(l1, 2000),
            nn.ReLU(),
            nn.Dropout(0.6),

            nn.Linear(2000, 200),
            nn.ReLU(),
            nn.Dropout(0.8),

            nn.Linear(200, 3),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.blackbox(x)
        return x

model = NeuralNetwork(l1).to(device)
print(model)

NeuralNetwork(
  (blackbox): Sequential(
    (0): Linear(in_features=17967, out_features=8000, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.6, inplace=False)
    (3): Linear(in_features=8000, out_features=4000, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.8, inplace=False)
    (6): Linear(in_features=4000, out_features=3, bias=True)
    (7): Softmax(dim=1)
  )
)


In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    loss_ls = []
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # Compute prediction error
        pred = model(X.float())
        loss = loss_fn(pred, y.long())
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(y)
            loss_ls.append(loss)
            
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    print(f"Avg train loss :{sum(loss_ls)/len(loss_ls)}")

def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X.float())
            test_loss += loss_fn(pred, y.long()).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg test loss: {test_loss:>8f} \n")

In [ ]:
# empty cuda memory
if 1:
    gc.collect()
    torch.cuda.empty_cache()
    t = 1
    lr = 1e-5

In [ ]:
lr=8e-6

In [ ]:
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.008, amsgrad=False)
epochs = 200

while t < epochs:
    print(f"Epoch {t}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
    
    if t%5 == 0:
        
        lr = lr * 0.99
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.008, amsgrad=False)
        print(f"Learning rate adjusted to {lr}")
        
        torch.save(model.state_dict(), path+f"nn/model_{t}.pt")
        print(f"Model saved to nn/model_{t}.pt")
    
    t += 1

print("Done!")

Epoch 7
-------------------------------
loss: 0.761930  [    0/43162]
loss: 0.703474  [ 3200/43162]
loss: 0.741789  [ 6400/43162]
loss: 0.736764  [ 9600/43162]
loss: 0.664158  [12800/43162]
loss: 0.677017  [16000/43162]
loss: 0.714896  [19200/43162]
loss: 0.725014  [22400/43162]
loss: 0.781269  [25600/43162]
loss: 0.665006  [28800/43162]
loss: 0.748388  [32000/43162]
loss: 0.809015  [35200/43162]
loss: 0.734407  [38400/43162]
loss: 0.701991  [41600/43162]
Avg train loss :0.7260797960417611
Test Error: 
 Accuracy: 80.4%, Avg test loss: 0.023221 

Epoch 8
-------------------------------
loss: 0.671219  [    0/43162]
loss: 0.715719  [ 3200/43162]
loss: 0.781397  [ 6400/43162]
loss: 0.701083  [ 9600/43162]
loss: 0.809983  [12800/43162]
loss: 0.795919  [16000/43162]
loss: 0.669350  [19200/43162]
loss: 0.812744  [22400/43162]
loss: 0.709863  [25600/43162]
loss: 0.649076  [28800/43162]
loss: 0.747459  [32000/43162]
loss: 0.663430  [35200/43162]
loss: 0.697412  [38400/43162]
loss: 0.681264  [4

KeyboardInterrupt: ignored

In [ ]:
X_test = scipy.sparse.load_npz(path+'data/countvec/test_steps_vec.npz').toarray().astype(np.float32)

In [ ]:
X_test = torch.tensor(X_test)

In [ ]:
testing_data = TensorDataset(X_t)

test_dataloader = DataLoader(testing_data, batch_size=64, shuffle=True)

In [ ]:
test_dataloader = DataLoader(testing_data, batch_size=64, shuffle=False)
pre = []
with torch.no_grad():
  for X0 in test_dataloader:
    X0 = X0[0]
    X0 = X0.to(device)
    pred = model(X0)
    pre.append(pred.argmax(1))

In [ ]:
pre = torch.cat([i for i in pre])

In [ ]:
pre = pre.cpu().numpy()+1

In [ ]:
pre = pre.astype('float')

In [ ]:
pre

array([1., 3., 1., ..., 2., 2., 3.])

In [ ]:
y_test = np.array(y_t)+1

In [ ]:
pred = pd.DataFrame(pre, index=range(1,10001), columns=['duration_label'])

In [ ]:
pred.to_csv("pred.csv", header=True, index=True)

In [ ]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, f1_score, accuracy_score

print(f"accu: {accuracy_score(y_test, pre)}")

precision = precision_score(y_test, pre, average=None, zero_division=0)
recall = recall_score(y_test, pre, average=None, zero_division=0)
f1 = f1_score(y_test, pre, average=None, zero_division=0)
score = pd.DataFrame({'Precision':precision, "Recall":recall, "F_score":f1}, index=[1,2,3])
print(score)
matrix = confusion_matrix(y_test, pre)
matrix = pd.DataFrame(matrix, index=[1,2,3], columns=[1,2,3])
print("\nConfusion matrix:")
print(matrix, end='\n\n')

accu: 0.7970530998053934
   Precision    Recall   F_score
1   0.763746  0.755310  0.759504
2   0.788554  0.760254  0.774145
3   0.841791  0.891277  0.865827

Confusion matrix:
      1     2     3
1  2667   616   248
2   700  3114   282
3   125   219  2820

